<a href="https://colab.research.google.com/github/clento173/OpenSourceDataAnalysis_3-2/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import requests
import pandas as pd
import json

url = "http://openapi.seoul.go.kr:8088/4d54555a6965756e3130326d6c4c5867/json/energyUseDataSummaryInfo/1/100/"
responses = []
for year in range(2015,2016):
    for month in range(1,13):
        url_month = url + str(year) + "/" + "{:02d}".format(month)
        responses.append(requests.get(url_month).json())

rows = responses[0]['energyUseDataSummaryInfo']['row']
select_col = [{'EUS' : item['EUS'], 'GUS' : item['GUS'], 'WUS' : item['WUS'], 'HUS' : item['HUS']} for item in rows]

print(select_col)


[{'EUS': '193784708', 'GUS': '59133720', 'WUS': '12819757.886', 'HUS': '22740838.937'}, {'EUS': '134955565', 'GUS': '9107197', 'WUS': '2075819.2', 'HUS': '0'}, {'EUS': '372270', 'GUS': '53331', 'WUS': '5945', 'HUS': '0'}, {'EUS': '10249618', 'GUS': '536119', 'WUS': '147273', 'HUS': '0'}, {'EUS': '327102096', 'GUS': '22834274', 'WUS': '12844807.5', 'HUS': '0'}, {'EUS': '583090528', 'GUS': '37835680', 'WUS': '6581017', 'HUS': '0'}, {'EUS': '247059373', 'GUS': '11524542', 'WUS': '1522465.2', 'HUS': '0'}]
